# Stock LSTM — Unified Backtest + 10‑day Forecast

This notebook is parameterized for Papermill.

**Inputs:** `TICKER, LOOKBACK, CONTEXT, BACKTEST_HORIZON, HORIZON, OUTPUT_JSON`

**Output file format (JSON)**:
```json
{
  "ticker": "AAPL",
  "look_back": 60,
  "context": 100,
  "backtest_horizon": 20,
  "horizon": 10,
  "metrics": {
    "rmse": 0.0,
    "mape": 0.0,
    "accuracy_pct": 0.0,
    "expected_10d_move_pct": 0.0
  },
  "forecast": [
    {"date": "YYYY-MM-DD", "actual": 123.45, "part": "context"},
    {"date": "YYYY-MM-DD", "pred": 123.45, "part": "backtest"},
    {"date": "YYYY-MM-DD", "pred": 123.45, "part": "forecast"}
  ]
}
```


In [10]:
#  imports
import os, math, json
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import tensorflow as tf
from joblib import dump, load
from pathlib import Path

print("TensorFlow:", tf.__version__)


TensorFlow: 2.20.0


In [11]:
# Papermill parameters (will be overridden when executed by Papermill)
TICKER = "AAPL"
LOOKBACK = 60
CONTEXT = 100
BACKTEST_HORIZON = 20
HORIZON = 10
OUTPUT_JSON = "forecast.json"
INTERVAL = "1d"

# Where to read a big training universe (one symbol per line)
TICKERS_FILE = "tickers_1000_plus.txt"

# Training / loading behavior
FORCE_RETRAIN_GLOBAL   = False   # True => retrain global model even if saved
ENABLE_CALIBRATOR      = True    # learn/apply per-ticker linear adjustment
AUTO_RETRAIN_ON_NEW    = True    # True => if TICKER wasn't in last global training, retrain now
RETRAIN_ADD_TO_UNIVERSE= True    # True => append new symbols to TICKERS_FILE before retraining

# Global artifacts (shared by ALL symbols)
ARTIFACTS_DIR   = Path("artifacts"); ARTIFACTS_DIR.mkdir(parents=True, exist_ok=True)
GLOBAL_MODEL_M  = ARTIFACTS_DIR / "global_lstm_multi.keras"   # HORIZON-step model
GLOBAL_MODEL_1  = ARTIFACTS_DIR / "global_lstm_one.keras"     # 1-step model
GLOBAL_SCALER_X = ARTIFACTS_DIR / "global_scaler_X.joblib"
GLOBAL_SCALER_Y = ARTIFACTS_DIR / "global_scaler_Y.joblib"    # for 1-step returns
REGISTRY_PATH   = ARTIFACTS_DIR / "symbol_index.json"
CALIBRATORS_JSON= ARTIFACTS_DIR / "calibrators.json"
TRAINED_SET_PATH= ARTIFACTS_DIR / "global_trained_symbols.json"  # snapshot of last global training universe

# Training universe to build the global model
if Path(TICKERS_FILE).exists():
    EXAMPLE_TICKERS = [l.strip() for l in Path(TICKERS_FILE).read_text().splitlines() if l.strip()]
else:
    EXAMPLE_TICKERS = [
        "SPY","QQQ","IWM","EFA","EEM","GLD","SLV","TLT","HYG","XLF","XLE","XLY","XLK","XLV","XLI","XLP","XLB","XLU","VNQ","ARKK",
        "AAPL","MSFT","GOOGL","AMZN","NVDA","META","TSLA","BRK-B","JPM","V","JNJ","WMT","PG","UNH","MA","HD","XOM","BAC","PFE","DIS",
        "BTC-USD","ETH-USD","SOL-USD","BNB-USD","XRP-USD","ADA-USD","DOGE-USD"
    ]
# Ensure the focus ticker is known in-memory (file update handled later if needed)
if TICKER not in EXAMPLE_TICKERS:
    EXAMPLE_TICKERS = [TICKER] + EXAMPLE_TICKERS


In [12]:
# -------------------------------
# Helper functions & GLOBAL model
# -------------------------------
MAX_SYMBOLS = 20000

def _load_registry():
    if REGISTRY_PATH.exists():
        try: return json.loads(REGISTRY_PATH.read_text())
        except Exception: pass
    return {"symbol_to_id":{}, "id_to_symbol":{}}

def _save_registry(reg:dict):
    REGISTRY_PATH.write_text(json.dumps(reg, indent=2))

def _ensure_sid(symbol:str, reg:dict)->int:
    s=symbol.upper().strip()
    if s in reg["symbol_to_id"]: return reg["symbol_to_id"][s]
    nxt = 0
    used = set(reg["symbol_to_id"].values())
    while nxt in used: nxt += 1
    reg["symbol_to_id"][s]=nxt; reg["id_to_symbol"][str(nxt)] = s
    _save_registry(reg)
    return nxt

def _load_calibrators():
    if CALIBRATORS_JSON.exists():
        try: return json.loads(CALIBRATORS_JSON.read_text())
        except Exception: pass
    return {}

def _save_calibrators(obj:dict):
    CALIBRATORS_JSON.write_text(json.dumps(obj, indent=2))

def was_in_last_training(symbol:str) -> bool:
    try:
        symbols = json.loads(TRAINED_SET_PATH.read_text())
        return symbol.upper() in {s.upper() for s in symbols}
    except Exception:
        return False

def add_symbol_to_universe_file(symbol:str, path=TICKERS_FILE):
    p = Path(path)
    lines = set()
    if p.exists():
        lines = {l.strip() for l in p.read_text().splitlines() if l.strip()}
    lines.add(symbol.upper())
    p.write_text("\n".join(sorted(lines)))

def fetch_prices(ticker: str, start="2016-01-01", end=None, interval="1d") -> pd.DataFrame:
    df = yf.download(ticker, start=start, end=end, interval=interval, auto_adjust=True, progress=False, timeout=30)
    if df is None or df.empty: raise ValueError("No data returned.")
    df.index = pd.to_datetime(df.index); df.index.name = "Date"
    cols = [c for c in ["Open","High","Low","Close","Volume"] if c in df.columns]
    if "Close" not in cols and "Adj Close" in df.columns:
        df["Close"] = df["Adj Close"]; cols = [*cols, "Close"]
    return df[cols].dropna()

def fetch_prices_auto(ticker: str, interval="1d") -> pd.DataFrame:
    for p in ["1y","150d","90d","60d","30d"]:
        try:
            df = yf.download(ticker, period=p, interval=interval, auto_adjust=True, progress=False, timeout=30)
            if df is not None and not df.empty:
                df = df[["Open","High","Low","Close","Volume"]].dropna()
                if not df.empty:
                    df.index = pd.to_datetime(df.index); df.index.name = "Date"
                    return df
        except Exception:
            pass
    raise ValueError("No data for any fallback period.")

def add_features(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    close = pd.to_numeric(out["Close"].squeeze(), errors="coerce").where(lambda x: x>0, np.nan)
    out["Close"] = close
    out["log_ret"] = np.log(close).diff()
    out["ret"] = close.pct_change()
    out["roll_mean_7"]  = close.rolling(7).mean()
    out["roll_std_7"]   = close.rolling(7).std()
    out["roll_mean_21"] = close.rolling(21).mean()
    out["roll_std_21"]  = close.rolling(21).std()
    delta = close.diff(); up = delta.clip(lower=0); down = -delta.clip(upper=0)
    roll_up = up.ewm(alpha=1/14, min_periods=14, adjust=False).mean()
    roll_dn = down.ewm(alpha=1/14, min_periods=14, adjust=False).mean()
    rs = roll_up / roll_dn.replace(0, np.nan)
    out["rsi_14"] = 100 - (100 / (1 + rs))
    ema12 = close.ewm(span=12, adjust=False).mean()
    ema26 = close.ewm(span=26, adjust=False).mean()
    macd = ema12 - ema26
    out["macd"] = macd
    out["macd_signal"] = macd.ewm(span=9, adjust=False).mean()
    out["macd_diff"] = out["macd"] - out["macd_signal"]
    ma20 = close.rolling(20).mean(); sd20 = close.rolling(20).std()
    out["bb_width"] = (ma20 + 2*sd20 - (ma20 - 2*sd20)) / close
    out["ret_lag1"] = out["log_ret"].shift(1)
    out["ret_lag3"] = out["log_ret"].shift(3)
    out["ret_lag5"] = out["log_ret"].shift(5)
    out["vol_7"] = out["log_ret"].rolling(7).std()
    out["vol_21"] = out["log_ret"].rolling(21).std()
    out["z_close_21"] = (close - close.rolling(21).mean()) / close.rolling(21).std()
    return out.dropna()

FEATURES = [
    "Close","Volume","log_ret","ret",
    "roll_mean_7","roll_std_7","roll_mean_21","roll_std_21",
    "rsi_14","macd","macd_signal","macd_diff",
    "bb_width","ret_lag1","ret_lag3","ret_lag5",
    "vol_7","vol_21","z_close_21",
]

def make_windows(X: np.ndarray, y: np.ndarray, L: int, H: int):
    xs, ys = [], []
    for i in range(L, len(X) - H + 1):
        xs.append(X[i-L:i, :])
        ys.append(y[i:i+H])
    return np.array(xs, dtype="float32"), np.array(ys, dtype="float32")

def build_global_model(input_steps: int, n_features: int, horizon: int) -> tf.keras.Model:
    ts_in  = tf.keras.Input(shape=(input_steps, n_features), name="ts_in")
    sid_in = tf.keras.Input(shape=(), dtype="int32", name="sid_in")
    emb = tf.keras.layers.Embedding(MAX_SYMBOLS, 16, name="sym_emb")(sid_in)
    emb_r = tf.keras.layers.RepeatVector(input_steps)(emb)
    x = tf.keras.layers.Concatenate()([ts_in, emb_r])
    x = tf.keras.layers.Conv1D(48, kernel_size=5, padding="causal", activation="relu")(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(160, return_sequences=True))(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    x = tf.keras.layers.LSTM(96)(x)
    out = tf.keras.layers.Dense(horizon, name="step_returns")(x)
    m = tf.keras.Model([ts_in, sid_in], out, name="global_lstm_multiasset")
    m.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss="mse")
    return m

def _fit_scaler_global(X: np.ndarray) -> StandardScaler:
    N,L,F = X.shape
    sc = StandardScaler().fit(X.reshape(N*L, F))
    return sc

def _apply_scaler_global(X: np.ndarray, sc: StandardScaler) -> np.ndarray:
    N,L,F = X.shape
    flat = sc.transform(X.reshape(N*L, F))
    return flat.reshape(N, L, F)

def _build_dataset_for_universe(tickers:list, lookback:int, horizon:int, interval:str="1d"):
    reg = _load_registry()
    Xs, ys, sids = [], [], []
    for s in tickers:
        try:
            df = fetch_prices_auto(s, interval=interval)
        except Exception:
            continue
        feat = add_features(df)
        X = feat[FEATURES].values.astype("float32")
        y = feat["ret"].fillna(0).values.astype("float32")
        Xw, Yw = make_windows(X, y, lookback, horizon)
        if len(Xw)==0: continue
        sid = _ensure_sid(s, reg)
        Xs.append(Xw)
        ys.append(Yw)
        sids.append(np.full((len(Xw),), sid, dtype="int32"))
    if not Xs:
        raise ValueError("No training samples from the provided universe.")
    X = np.concatenate(Xs, axis=0)
    Y = np.concatenate(ys, axis=0)
    S = np.concatenate(sids, axis=0)
    return X, Y, S, reg

def ensure_global_trained(lookback:int, horizon:int, interval:str="1d", force:bool=False):
    need_train = force or (not GLOBAL_MODEL_M.exists()) or (not GLOBAL_MODEL_1.exists()) or (not GLOBAL_SCALER_X.exists()) or (not GLOBAL_SCALER_Y.exists())
    if not need_train:
        return
    X, Y, S, reg = _build_dataset_for_universe(EXAMPLE_TICKERS, lookback, horizon, interval=interval)
    scX = _fit_scaler_global(X)
    Xs  = _apply_scaler_global(X, scX)
    # multi-step model
    mM = build_global_model(lookback, Xs.shape[-1], horizon)
    mM.fit({"ts_in":Xs,"sid_in":S}, Y, epochs=18, batch_size=256, verbose=1)
    mM.save(GLOBAL_MODEL_M)
    dump(scX, GLOBAL_SCALER_X)
    _save_registry(reg)
    # one-step model
    Y1 = Y[:, :1]
    m1 = build_global_model(lookback, Xs.shape[-1], 1)
    m1.fit({"ts_in":Xs,"sid_in":S}, Y1, epochs=12, batch_size=256, verbose=1)
    m1.save(GLOBAL_MODEL_1)
    scY = StandardScaler().fit(Y1.reshape(-1,1))
    dump(scY, GLOBAL_SCALER_Y)
    # snapshot which symbols trained the current global weights
    TRAINED_SET_PATH.write_text(json.dumps(sorted(list(set(EXAMPLE_TICKERS))), indent=2))

def infer_freq_has_weekends(idx: pd.Index) -> bool:
    try:
        wd = pd.Index([pd.Timestamp(x).weekday() for x in idx])
        return (wd >= 5).any()
    except Exception:
        return False

def next_dates(last_date, n: int, use_weekends: bool) -> list:
    last_ts = pd.Timestamp(last_date)
    if use_weekends:
        rng = pd.date_range(last_ts, periods=n+1, freq="D"); return [d for d in rng[1:]]
    else:
        rng = pd.bdate_range(last_ts, periods=n+1); return [d for d in rng[1:]]


In [13]:
# -------------------------------
# Main logic (GLOBAL model + per-ticker calibrator)
# -------------------------------

# 0) If this is a brand-new symbol vs last global training, optionally retrain now
is_new_to_global = not was_in_last_training(TICKER)
if AUTO_RETRAIN_ON_NEW and is_new_to_global:
    if RETRAIN_ADD_TO_UNIVERSE:
        add_symbol_to_universe_file(TICKER, path=TICKERS_FILE)
    if TICKER not in EXAMPLE_TICKERS:
        EXAMPLE_TICKERS = [TICKER] + EXAMPLE_TICKERS
    ensure_global_trained(LOOKBACK, HORIZON, interval=INTERVAL, force=True)
else:
    ensure_global_trained(LOOKBACK, HORIZON, interval=INTERVAL, force=FORCE_RETRAIN_GLOBAL)

# 1) Pull enough data for the single target TICKER (smart fallback)
buffer_days = 320
start_date = (pd.Timestamp.utcnow() - pd.Timedelta(days=CONTEXT + LOOKBACK + BACKTEST_HORIZON + buffer_days)).date().isoformat()
try:
    raw = fetch_prices(TICKER, start=start_date, interval=INTERVAL)
    df = add_features(raw)
    if len(df) < (LOOKBACK + BACKTEST_HORIZON + CONTEXT + 10):
        raise ValueError("fallback")
except Exception:
    raw = fetch_prices_auto(TICKER, interval=INTERVAL)
    df = add_features(raw)

df["target_ret"] = df["log_ret"].shift(-1)
df = df.dropna()
if len(df) < LOOKBACK + BACKTEST_HORIZON + 5:
    raise ValueError("Not enough rows after feature engineering.")

# 2) Keep just the window we need for context + train + backtest slice (for calibrator)
df_tail = df.tail(CONTEXT + LOOKBACK + BACKTEST_HORIZON)
X_df = df_tail[FEATURES].astype("float32")
y = df_tail["target_ret"].astype("float32").values
dates = df_tail.index

# split point for backtest
train_end_idx = len(df_tail) - BACKTEST_HORIZON
if train_end_idx <= LOOKBACK:
    train_end_idx = LOOKBACK + 1
    BACKTEST_HORIZON = max(1, len(df_tail) - train_end_idx)

X_train_df = X_df.iloc[:train_end_idx]
y_train = y[:train_end_idx]
n_features = X_train_df.shape[1]

# 3) Load global scalers/models
scaler_X = load(GLOBAL_SCALER_X)
scaler_Y = load(GLOBAL_SCALER_Y)
m_multi  = tf.keras.models.load_model(GLOBAL_MODEL_M)
m_one    = tf.keras.models.load_model(GLOBAL_MODEL_1)

# Symbol id for the target ticker
reg = _load_registry()
sid = _ensure_sid(TICKER, reg)

# 4) Backtest last BACKTEST_HORIZON days (1-step each) to update per-ticker calibrator
backtest_dates, backtest_pred_prices, backtest_actual_prices, backtest_pred_rets = [], [], [], []
close_np = df_tail["Close"].to_numpy()

for k in range(int(BACKTEST_HORIZON)):
    end_idx = train_end_idx + k
    xb_raw = X_df.values[end_idx-LOOKBACK:end_idx, :]
    xb = scaler_X.transform(xb_raw).reshape(1, LOOKBACK, n_features)
    y1_s = m_one.predict({"ts_in":xb, "sid_in":np.array([sid])}, verbose=0)[0]  # (1,)
    pred_ret = scaler_Y.inverse_transform(y1_s.reshape(-1,1))[0,0]
    last_price   = float(close_np[end_idx-1])
    pred_price   = float(last_price * np.exp(pred_ret))
    actual_price = float(close_np[end_idx])
    backtest_dates.append(pd.Timestamp(dates[end_idx]).strftime("%Y-%m-%d"))
    backtest_pred_prices.append(pred_price)
    backtest_actual_prices.append(actual_price)
    backtest_pred_rets.append(float(pred_ret))

rmse = float(np.sqrt(mean_squared_error(backtest_actual_prices, backtest_pred_prices)))
mape = float(mean_absolute_percentage_error(backtest_actual_prices, backtest_pred_prices) * 100)

actual_rets = []
for k in range(int(BACKTEST_HORIZON)):
    prev_price = float(close_np[train_end_idx - 1 + k])
    cur_price  = float(close_np[train_end_idx + k])
    actual_rets.append((cur_price - prev_price) / (prev_price if prev_price != 0 else 1.0))
acc = float((np.sign(backtest_pred_rets) == np.sign(actual_rets)).mean() * 100.0) if backtest_pred_rets and actual_rets else 0.0

# 5) Multi-step forward forecast of next HORIZON sessions + optional per-ticker calibrator
last_block_raw = X_df.values[-LOOKBACK:]
last_block = scaler_X.transform(last_block_raw).reshape(1, LOOKBACK, n_features)
next_rets_s = m_multi.predict({"ts_in":last_block, "sid_in":np.array([sid])}, verbose=0)[0]  # (HORIZON,)
next_rets = next_rets_s.astype("float64")  # multi-step model learned raw returns

if ENABLE_CALIBRATOR:
    key = TICKER.upper()
    cals = _load_calibrators()
    if len(backtest_pred_rets) >= 5:
        A = np.vstack([backtest_pred_rets, np.ones(len(backtest_pred_rets))]).T
        sol, *_ = np.linalg.lstsq(A, np.array(actual_rets[:len(backtest_pred_rets)]), rcond=None)
        a, b = float(sol[0]), float(sol[1])
        cals[key] = {"a": a, "b": b}
        _save_calibrators(cals)
    if key in cals:
        a, b = cals[key]["a"], cals[key]["b"]
        next_rets = a * next_rets + b

last_price = float(close_np[-1])
future_pred_prices = (last_price * np.exp(np.cumsum(next_rets))).astype(float)
expected_move_pct = float((float(future_pred_prices[-1]) - last_price) / (last_price if last_price != 0 else 1.0) * 100.0)

# 6) Build unified series for frontend (context actuals + backtest preds + future preds)
series_map = {}
ctx_tail = df_tail.tail(int(CONTEXT))
for d, p in zip(ctx_tail.index, ctx_tail["Close"].astype(float).values):
    ds = pd.Timestamp(d).strftime("%Y-%m-%d")
    row = series_map.get(ds, {"date": ds}); row["actual"] = float(p); row.setdefault("part", "context"); series_map[ds] = row

for i, dstr in enumerate(backtest_dates[:len(backtest_pred_prices)]):
    row = series_map.get(dstr, {"date": dstr}); row["pred"] = float(backtest_pred_prices[i]); row["part"] = "backtest"; series_map[dstr] = row

has_weekends = infer_freq_has_weekends(df_tail.index)
future_dates = next_dates(df_tail.index[-1], int(HORIZON), use_weekends=has_weekends)
for i, p in enumerate(future_pred_prices):
    ds = pd.Timestamp(future_dates[i]).strftime("%Y-%m-%d")
    row = series_map.get(ds, {"date": ds}); row["pred"] = float(p); row["part"] = "forecast"; series_map[ds] = row

series = sorted(series_map.values(), key=lambda r: r["date"])
metrics = {"rmse": float(rmse), "mape": float(mape), "accuracy_pct": float(acc), "expected_10d_move_pct": float(expected_move_pct)}
result = {
  "ticker": str(TICKER).upper(),
  "interval": INTERVAL,
  "look_back": int(LOOKBACK),
  "context": int(CONTEXT),
  "backtest_horizon": int(BACKTEST_HORIZON),
  "horizon": int(HORIZON),
  "metrics": metrics,
  "forecast": series,
}

out_path = str(OUTPUT_JSON)
os.makedirs(os.path.dirname(out_path) or ".", exist_ok=True)
with open(out_path, "w") as f:
    json.dump(result, f)
print("WROTE JSON =>", out_path)
print("In last global training set:", was_in_last_training(TICKER))
print("Has calibrator now:", TICKER.upper() in _load_calibrators())



1 Failed download:
['ATVI']: YFPricesMissingError('possibly delisted; no price data found  (period=1y) (Yahoo error = "No data found, symbol may be delisted")')

1 Failed download:
['ATVI']: YFPricesMissingError('possibly delisted; no price data found  (period=150d) (Yahoo error = "No data found, symbol may be delisted")')

1 Failed download:
['ATVI']: YFPricesMissingError('possibly delisted; no price data found  (period=90d) (Yahoo error = "No data found, symbol may be delisted")')

1 Failed download:
['ATVI']: YFPricesMissingError('possibly delisted; no price data found  (period=60d) (Yahoo error = "No data found, symbol may be delisted")')

1 Failed download:
['ATVI']: YFPricesMissingError('possibly delisted; no price data found  (period=30d) (Yahoo error = "No data found, symbol may be delisted")')

1 Failed download:
['PEAK']: YFPricesMissingError('possibly delisted; no price data found  (period=1y) (Yahoo error = "No data found, symbol may be delisted")')

1 Failed download:
['P

Epoch 1/18
285/285 ━━━━━━━━━━━━━━━━━━━━ 48s 162ms/step - loss: 971452.2500
Epoch 2/18
285/285 ━━━━━━━━━━━━━━━━━━━━ 49s 170ms/step - loss: 971427.1250
Epoch 3/18
285/285 ━━━━━━━━━━━━━━━━━━━━ 51s 180ms/step - loss: 971411.8125
Epoch 4/18
285/285 ━━━━━━━━━━━━━━━━━━━━ 51s 179ms/step - loss: 971398.0625
Epoch 5/18
285/285 ━━━━━━━━━━━━━━━━━━━━ 52s 182ms/step - loss: 971397.1875
Epoch 6/18
285/285 ━━━━━━━━━━━━━━━━━━━━ 52s 184ms/step - loss: 971392.5625
Epoch 7/18
285/285 ━━━━━━━━━━━━━━━━━━━━ 53s 186ms/step - loss: 971377.1250
Epoch 8/18
285/285 ━━━━━━━━━━━━━━━━━━━━ 54s 188ms/step - loss: 971365.8750
Epoch 9/18
285/285 ━━━━━━━━━━━━━━━━━━━━ 54s 189ms/step - loss: 971358.4375
Epoch 10/18
285/285 ━━━━━━━━━━━━━━━━━━━━ 56s 198ms/step - loss: 971344.6250
Epoch 11/18
285/285 ━━━━━━━━━━━━━━━━━━━━ 56s 198ms/step - loss: 971338.3125
Epoch 12/18
285/285 ━━━━━━━━━━━━━━━━━━━━ 56s 198ms/step - loss: 971330.3125
Epoch 13/18
285/285 ━━━━━━━━━━━━━━━━━━━━ 57s 199ms/step - loss: 971318.3750
Epoch 14/18
285/285 ━

/var/folders/vx/b0mk9dxj6cj0p22ynsbn1b440000gn/T/ipykernel_4843/79452759.py:69: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  last_price   = float(close_np[end_idx-1])
/var/folders/vx/b0mk9dxj6cj0p22ynsbn1b440000gn/T/ipykernel_4843/79452759.py:71: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  actual_price = float(close_np[end_idx])
/var/folders/vx/b0mk9dxj6cj0p22ynsbn1b440000gn/T/ipykernel_4843/79452759.py:69: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  last_pric

WROTE JSON => forecast.json
In last global training set: True
Has calibrator now: True


/var/folders/vx/b0mk9dxj6cj0p22ynsbn1b440000gn/T/ipykernel_4843/79452759.py:106: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  last_price = float(close_np[-1])
/var/folders/vx/b0mk9dxj6cj0p22ynsbn1b440000gn/T/ipykernel_4843/79452759.py:115: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  row = series_map.get(ds, {"date": ds}); row["actual"] = float(p); row.setdefault("part", "context"); series_map[ds] = row
